# Trung Nguyen - 111752939
## Assignment #3: Data Integration and Modeling 

## Task 1 (10%)

> - Build a scoring function to rank houses by “desirability”, presumably a notion related to cost or value.  
- Identify what the ten most desirable and least desirable houses in the Kaggle data set are.
- Write a one page description of which variables your function used and how well you think it worked. 

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression


df = pd.read_csv('./data/properties_2017.csv', low_memory=False)
# Calculate mean value and replace NaN values with mean value

There are fields whose values are numercal but they should be listed as categorical, since their numerical values does not reflex the level of affection to the model (Such as ``typeid`` fields) 

Changing each of their value to a new binary feature would be the right way to extract information from them, but then the model becomes very complex with too many features, so I simply drop them.

I also drop all binary fields, as I don't have experience with handling both categorical features and numerical features in the same model.

I also fill the NaN values with 0 to avoid dealing with NaN values in finding the most positive and negative correlations

(Actually I think it would be better to fill the matrix with median values. I tried to fill all the fields with its median values, however, the computation was too much that my laptop couldn't handle it)

In [13]:
df_dropped = df.drop(["hashottuborspa", "propertycountylandusecode", "propertyzoningdesc", 
                            "fireplaceflag", "taxdelinquencyflag", "assessmentyear", 
                            "airconditioningtypeid", "buildingclasstypeid", "decktypeid",
                            "heatingorsystemtypeid", "regionidcity", "regionidcounty",
                            "poolcnt", "fips", "buildingqualitytypeid", "threequarterbathnbr",
                            "regionidneighborhood", "regionidzip", "storytypeid",
                            "pooltypeid7", "rawcensustractandblock", "censustractandblock",
                            "typeconstructiontypeid", "pooltypeid10", "pooltypeid2", "architecturalstyletypeid",
                            "bathroomcnt", "propertylandusetypeid"], axis=1)
df_dropped_filled0 = df_dropped.fillna(0)


In [14]:
df_dropped_filled0.shape

(2985217, 30)

In [15]:
df_dropped_filled0.dtypes

parcelid                          int64
basementsqft                    float64
bedroomcnt                      float64
calculatedbathnbr               float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
poolsizesum                     float64
roomcnt                         float64
unitcnt                         float64
yardbuildingsqft17              float64
yardbuildingsqft26              float64
yearbuilt                       float64
numberofstories                 float64


## Task 2 (10%)

> - Define a house “pairwise distance function”, which measures the similarity of two properties.  Like a distance metric, similar pairs of very similar properties should be distance near zero, with distance increasing as the properties grow more dissimilar.  
- Experiment with your distance function, and write a one page discussion evaluating how well you think it worked.   Your function should include geographic as well as property-specific variables. 


## Task 3 (15%)

> - Using your distance function and an appropriate clustering algorithm, cluster the houses using your distance function into 10 to 100 classes, as you see best.   
- Present a dot-plot/map (with tiny dots colored to reflect the clustering) illustrating the clusters your method produced.    
- Write a one page discussion/analysis of what your clusters seem to be capturing, and how well they work 



## Task 4 (20%)

> - Identify at least one external data set which you can integrate into your price prediction analysis to make it better.  Perhaps it can be financial, such as the historical effects of interest rates, consumer confidence, etc. on housing prices.  Perhaps it can be geographic, like the crime rate, educational scores, income levels, etc.   
- Write a one page discussion/analysis on whether this data helps with the prediction tasks 




## Task 5 (20%)

> - Finally, build the best prediction model you can to solve the Zillow task.  Use any data, ideas, and approach that you like. Predict the logerror for instances at file “sample_submission.csv”.  
- Report the score/rank you get. 
- Write a 2-3 page report about how it works, an evaluation, and any interesting experiences along the way.  



## Task 6 (15%)

> - Do a permutation test to determine a p-value of how good your predictions of logerror are.  You can use whatever metric you wish to score your model (like mean absolute error).   
- For a large enough sample of the evaluation data, compare how your model ranks by this metric on the real data compared to 100 (or more) random permutations of the logerror assigned to the real data records. 
- What fraction of permutations produce at least error at least as good at the real data set?   If necessary, sample your data so these 100+ runs do not take too much time. 




## Task 7 (10%)

> Submit your results on the real test data to Kaggle before deadline. Write the result into a csv file and submit it to the website. Actually, submit two for your two best models, to the extent that Zillow allows it.   


